In [87]:
from ast import literal_eval
import logging
import os

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import evaluate
import imagesize
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

from src.full_model.evaluate_bbox_variations.custom_dataset_bbox_variations import CustomDatasetBboxVariations
from src.full_model.report_generation_model import ReportGenerationModel
from src.full_model.train_full_model import get_tokenizer
from src.path_datasets_and_weights import path_runs_full_model

# specify the checkpoint you want to evaluate by setting "RUN" and "CHECKPOINT"
RUN = 46
CHECKPOINT = "checkpoint_val_loss_19.793_overall_steps_155252.pt"
IMAGE_INPUT_SIZE = 512
BATCH_SIZE = 8
NUM_BEAMS = 4
MAX_NUM_TOKENS_GENERATE = 300

# test csv file with only 1000 images (you can create it by setting NUM_ROWS_TO_CREATE_IN_NEW_CSV_FILES in line 67 of create_dataset.py to 1000)
path_to_partial_test_set = "/u/home/tanida/datasets/dataset-with-reference-reports-partial/test-200.csv"

# path where "bbox_variations_results.txt" will be saved
path_results_txt_file = "/u/home/tanida/region-guided-chest-x-ray-report-generation/src/full_model/evaluate_bbox_variations/bbox_variations_results.txt"

In [88]:
def get_test_set_as_df():
    def compute_bbox_widths_heights(row):
        bbox_coordinates_single_image = row["bbox_coordinates"]
        widths_heights = []
        for bbox_coords in bbox_coordinates_single_image:
            x1, y1, x2, y2 = bbox_coords
            width = x2 - x1
            height = y2 - y1
            widths_heights.append([width, height])

        return widths_heights

    def retrieve_image_widths_heights(row):
        mimic_image_file_path = row["mimic_image_file_path"]
        width, height = imagesize.get(mimic_image_file_path)
        return [width, height]

    usecols = [
        "mimic_image_file_path",
        "bbox_coordinates",
        "bbox_labels",
        "bbox_phrases"
    ]

    # all of the columns below are stored as strings in the csv_file
    # however, as they are actually lists, we apply the literal_eval func to convert them to lists
    converters = {
        "bbox_coordinates": literal_eval,
        "bbox_labels": literal_eval,
        "bbox_phrases": literal_eval
    }

    test_set_as_df = pd.read_csv(path_to_partial_test_set, usecols=usecols, converters=converters)

    # add new columns that contain the bbox_widths_heights (List[List[int]] with len(outer_list)=29 and len(inner_list) = 2)
    # and image_width_height (List[int] of len 2)
    test_set_as_df["bbox_widths_heights"] = test_set_as_df.apply(lambda row: compute_bbox_widths_heights(row), axis=1)
    test_set_as_df["image_width_height"] = test_set_as_df.apply(lambda row: retrieve_image_widths_heights(row), axis=1)

    return test_set_as_df

In [89]:
test_set_as_df = get_test_set_as_df()

In [90]:
tokenizer = get_tokenizer()

In [91]:
num_images = len(test_set_as_df)

In [92]:
mean = 0
std = 0.5

In [93]:
def get_transforms():
    # see compute_mean_std_dataset.py in src/dataset
    mean = 0.471
    std = 0.302

    # don't apply data augmentations to test set
    test_transforms = A.Compose(
        [
            A.LongestMaxSize(max_size=IMAGE_INPUT_SIZE, interpolation=cv2.INTER_AREA),
            A.PadIfNeeded(min_height=IMAGE_INPUT_SIZE, min_width=IMAGE_INPUT_SIZE, border_mode=cv2.BORDER_CONSTANT),
            A.Normalize(mean=mean, std=std),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"]),
    )

    return test_transforms

In [94]:
transforms = get_transforms()

In [95]:
scale_variations = np.exp(np.random.normal(mean, std, size=(num_images, 29)))

In [96]:
test_set_as_df["scale_variations"] = scale_variations.tolist()

In [97]:
test_set_as_df.head()

,mimic_image_file_path,bbox_coordinates,bbox_labels,bbox_phrases,bbox_widths_heights,image_width_height,scale_variations
0,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[327, 231, 1200, 2114], [477, 300, 1200, 968]...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[There is no focal consolidation, pleural effu...","[[873, 1883], [723, 668], [751, 423], [819, 72...","[2544, 3056]","[0.6870780669617994, 1.9312716314575513, 1.864..."
1,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[300, 382, 1227, 2332], [477, 436, 1227, 1118...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[Pulmonary vasculature is normal. Lungs are cl...,"[[927, 1950], [750, 682], [778, 437], [859, 77...","[2544, 3056]","[1.101386192840625, 1.1908239541509842, 1.3439..."
2,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[229, 652, 1171, 2330], [386, 676, 1086, 1135...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[No acute intrathoracic process. There is no f...,"[[942, 1678], [700, 459], [870, 434], [942, 76...","[2539, 2705]","[0.9995962823161972, 0.726682867938654, 2.2196..."
3,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[467, 596, 1453, 2451], [505, 648, 1206, 1154...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[No acute intrathoracic process. There is no f...,"[[986, 1855], [701, 506], [817, 467], [973, 83...","[2258, 2906]","[1.850598179298774, 0.7658447936256262, 0.8453..."
4,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[286, 341, 1255, 2073], [450, 368, 1255, 1036...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[The lungs are clear of focal consolidation, p...","[[969, 1732], [805, 668], [846, 369], [873, 66...","[2544, 3056]","[1.3319810149528313, 1.172622694511421, 0.9614..."


In [98]:
def check_coordinate(coord, dimension):
    """Make sure that new (varied) coordinate is still within the image."""
    if coord < 0:
        return 0
    elif coord > dimension:
        return dimension
    else:
        return coord


def vary_bbox_coords_by_scale(row):
    bbox_coords_single_image = row["bbox_coordinates"]  # List[List[int]] of shape 29 x 4
    bbox_widths_heights_single_image = row["bbox_widths_heights"]  # List[List[int]] of shape 29 x 2
    scale_variations_bboxes = row["scale_variations"]  # List[float] of len 29
    image_width, image_height = row["image_width_height"]  # two integers

    # to store the new bbox coordinates after they have been varied
    varied_bbox_coords_single_image = []

    for bbox_coords, bbox_width_height, scale_variations in zip(bbox_coords_single_image, bbox_widths_heights_single_image, scale_variations_bboxes):
        x1, y1, x2, y2 = bbox_coords
        bbox_width, bbox_height = bbox_width_height
        scale_factor = scale_variations  # single positive float, e.g. 1.232 or 0.845

        # gt_bbox_mid_point stays the same for the scaled bbox, so it serves as the "anchor point" to compute the new bbox coordinates (using the new bbox width and height)
        ground_truth_bbox_mid_point_x = x1 + bbox_width / 2
        ground_truth_bbox_mid_point_y = y1 + bbox_height / 2

        bbox_width_scaled = bbox_width * scale_factor
        bbox_height_scaled = bbox_height * scale_factor

        x1_new = ground_truth_bbox_mid_point_x - bbox_width_scaled / 2
        x2_new = ground_truth_bbox_mid_point_x + bbox_width_scaled / 2

        y1_new = ground_truth_bbox_mid_point_y - bbox_height_scaled / 2
        y2_new = ground_truth_bbox_mid_point_y + bbox_height_scaled / 2

        x1 = check_coordinate(int(x1_new), image_width)
        x2 = check_coordinate(int(x2_new), image_width)
        y1 = check_coordinate(int(y1_new), image_height)
        y2 = check_coordinate(int(y2_new), image_height)

        varied_bbox_coords_single_image.append([x1, y1, x2, y2])

    return varied_bbox_coords_single_image

In [99]:
variation_func = vary_bbox_coords_by_scale

In [100]:
test_set_as_df["bbox_coordinates_varied"] = test_set_as_df.apply(lambda row: variation_func(row), axis=1)

In [101]:
test_set_as_df.head()

,mimic_image_file_path,bbox_coordinates,bbox_labels,bbox_phrases,bbox_widths_heights,image_width_height,scale_variations,bbox_coordinates_varied
0,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[327, 231, 1200, 2114], [477, 300, 1200, 968]...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[There is no focal consolidation, pleural effu...","[[873, 1883], [723, 668], [751, 423], [819, 72...","[2544, 3056]","[0.6870780669617994, 1.9312716314575513, 1.864...","[[463, 525, 1063, 1819], [140, 0, 1536, 1279],..."
1,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[300, 382, 1227, 2332], [477, 436, 1227, 1118...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[Pulmonary vasculature is normal. Lungs are cl...,"[[927, 1950], [750, 682], [778, 437], [859, 77...","[2544, 3056]","[1.101386192840625, 1.1908239541509842, 1.3439...","[[253, 283, 1273, 2430], [405, 370, 1298, 1183..."
2,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[229, 652, 1171, 2330], [386, 676, 1086, 1135...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[No acute intrathoracic process. There is no f...,"[[942, 1678], [700, 459], [870, 434], [942, 76...","[2539, 2705]","[0.9995962823161972, 0.726682867938654, 2.2196...","[[229, 652, 1170, 2329], [481, 738, 990, 1072]..."
3,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[467, 596, 1453, 2451], [505, 648, 1206, 1154...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[No acute intrathoracic process. There is no f...,"[[986, 1855], [701, 506], [817, 467], [973, 83...","[2258, 2906]","[1.850598179298774, 0.7658447936256262, 0.8453...","[[47, 0, 1872, 2906], [587, 707, 1123, 1094], ..."
4,/u/home/tanida/datasets/mimic-cxr-jpg/files/p1...,"[[286, 341, 1255, 2073], [450, 368, 1255, 1036...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[The lungs are clear of focal consolidation, p...","[[969, 1732], [805, 668], [846, 369], [873, 66...","[2544, 3056]","[1.3319810149528313, 1.172622694511421, 0.9614...","[[125, 53, 1415, 2360], [380, 310, 1324, 1093]..."
